In [ ]:
print("Okay")

In [ ]:
import os
os.makedirs("data", exist_ok=True)
print("Created data/ folder")


In [ ]:
import os
print("Current directory:", os.getcwd())
print("Files here:", os.listdir("."))


In [ ]:
import os 
os.chdir("../")

In [ ]:
%pwd

In [ ]:
import langchain; print(langchain.__version__)

In [ ]:
pip list | findstr langchain

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from pinecone import Pinecone as pinecone  # NEW Pinecone 8.0 import
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers

print("✅ ALL IMPORTS SUCCESS!")


In [ ]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf(data='data/')

In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data) 
print("Number of text chunks:", len(text_chunks))

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
!pip install langchain-groq


In [ ]:
import os
from dotenv import load_dotenv  # ← MISSING LINE!

# CRITICAL: Load .env file FIRST
load_dotenv()  # ← ADD THIS LINE

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

print("PINECONE_API_KEY:", "Set" if PINECONE_API_KEY else "MISSING")
print("GROQ_API_KEY:", "Set" if GROQ_API_KEY else "MISSING")


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medibuddy-trials"

pc.create_index(
    name=index_name,  
    dimension=384,  # Dimension of the embedding model
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


In [ ]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY





In [ ]:
from pinecone import Pinecone as pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings

# YOUR VALUES HERE
API_KEY = "pcsk_Xtpo7_NgZ8nAnKBMme8Meqi7FnRqhmgXYqb8ite55K7RA2LURZoZa59sdeamR9kGyb86k"  # From Pinecone dashboard
INDEX_NAME = "medibuddy-trials"     # From Pinecone dashboard (e.g., "medibuddy-docs")

# Connect to Pinecone
pc = pinecone(api_key=API_KEY)
index = pc.Index(INDEX_NAME)

# Create embeddings model (384 dimensions = matches your index)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

print("✅ Pinecone + Embeddings ready!")


In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load all PDFs from folder (CHANGE PATH)
loader = DirectoryLoader("./data", glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200
)
text_chunks = text_splitter.split_documents(documents)

print(f"✅ Created {len(text_chunks)} chunks!")


In [ ]:
from langchain_pinecone import PineconeVectorStore

# THIS WORKS NOW - all variables defined above
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

print("✅ Documents embedded and stored in Pinecone!")


In [ ]:
#Load Existing index
from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
     index_name=INDEX_NAME,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retriever_docs = retriever.invoke("What is Acne?")

In [ ]:
retriever_docs

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

# FASTER + SMARTER model for RAG
pipe = pipeline(
    "text-generation",
    model="distilgpt2",
    device=-1,  # CPU
    max_new_tokens=100,  # Short = fast
    temperature=0.1,     # Less random
    repetition_penalty=1.2,  # Stop "skin skin skin"
    pad_token_id=50256
)
llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
!pip install groq


In [77]:
from dotenv import load_dotenv
import os
from groq import Groq
load_dotenv()

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

def medibuddy_groq(query):
    """Production Medibuddy with "I don't know" safety"""
    docs = retriever.invoke(query)
    
    # SAFETY CHECK: If no relevant medical docs found
    if not docs or len(docs) == 0 or all(len(doc.page_content.strip()) < 50 for doc in docs):
        return "**🩺 Medibuddy:** I don't have information about that topic in my medical encyclopedia. Please ask about medical conditions, treatments, or symptoms."
    
    context = "\n".join([doc.page_content[:800] for doc in docs[:3]])
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system", 
                "content": """You are Medibuddy, a professional medical assistant. 
                Answer using ONLY the medical encyclopedia context provided. 
                Give concise, accurate answers in 2-3 sentences.
                If the context doesn't contain relevant medical information, say "I don't know." """
            },
            {
                "role": "user", 
                "content": f"CONTEXT:\n{context}\n\nQUESTION: {query}"
            }
        ],
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_tokens=300
    )
    
    return f"**🩺 Medibuddy:** {chat_completion.choices[0].message.content.strip()}"

# TEST SAFETY FEATURES!
print(medibuddy_groq("What is Acetaminophen?"))           # ✅ Medical answer
print(medibuddy_groq("Acne treatment"))          # ✅ Medical answer
print(medibuddy_groq("How to fix my laptop?"))   # ✅ "I don't know"
print(medibuddy_groq("Weather today"))           # ✅ "I don't know"
print(medibuddy_groq("Python programming"))      # ✅ "I don't know"


**🩺 Medibuddy:** Acetaminophen is a medicine used to relieve pain and reduce fever. It is available without a prescription and is sold under various brand names, including Tylenol and Panadol. Acetaminophen relieves pain and reduces fever about as well as aspirin, but is less likely to irritate the stomach.
**🩺 Medibuddy:** Acne treatment consists of reducing sebum production, removing dead skin cells, and killing bacteria with topical drugs and oral medications. The choice of treatment depends on the severity of the acne, which can be mild, moderate, or severe. Topical drugs, such as isotretinoin, are often used to treat mild non-inflammatory acne.
**🩺 Medibuddy:** I don't know.
**🩺 Medibuddy:** I don't know. The provided context does not contain information about the current weather. It discusses byssinosis, tricyclic antidepressants, and allergic rhinitis, but does not mention weather conditions.
**🩺 Medibuddy:** I don't know.
